## 1. Import Pakages

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

## 2. 하이퍼파라미터 세팅

In [ ]:
batch_size = 100
num_epochs = 300
learning_rate = 0.0002
z_size = 50

## 3. Dataset 및 DataLoader 정의

In [ ]:
root = '../data/mnist'
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
train_data = dset.MNIST(root=root, train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(
                 dataset=train_data,
                 batch_size=batch_size,
                 shuffle=True)

## 4. 데이터 시각화

In [ ]:
columns = 5
rows = 5
fig = plt.figure(figsize=(8,8))

for i in range(1, columns*rows+1):
    data_idx = np.random.randint(len(train_data))
    img = train_data[data_idx][0][0,:,:].numpy() # numpy()를 통해 torch Tensor를 numpy array로 변환
    label = train_data[data_idx][1].item() # item()을 통해 torch Tensor를 숫자로 변환
    
    fig.add_subplot(rows, columns, i)
    plt.title(label)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
plt.show()

## 5. 네트워크 설계

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(z_size + 10, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 28 * 28),
            nn.Tanh()
        )

    def forward(self, z, condition):
        concat = torch.cat((z, condition), dim=1)
        img = self.model(concat)
        img = img.view(-1, 1, 28, 28)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28 * 28 + 10, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img, condition):
        img = img.view(img.shape[0], -1)
        concat = torch.cat((img, condition), dim=1)
        validity = self.model(concat)
        return validity

## 6. 모델 생성 및 loss function, optimizer 정의

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
generator = Generator().to(device)
discriminator = Discriminator().to(device)
adversarial_loss = torch.nn.BCELoss()
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

## 7. Training

In [ ]:
for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        valid = torch.ones(batch_size, 1).to(device)
        fake = torch.zeros(batch_size, 1).to(device)
        real_imgs = imgs.to(device)
        z = nn.init.normal(torch.Tensor(batch_size,z_size),mean=0,std=0.1).to(device)
        
        condition = torch.FloatTensor(batch_size, 10).to(device)
        condition.zero_()
        labels = torch.unsqueeze(labels, 1).to(device)
        condition.scatter_(1, labels, 1)
        
        gen_imgs = generator(z, condition)
        
        g_loss = adversarial_loss(discriminator(gen_imgs, condition), valid)
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

        real_loss = adversarial_loss(discriminator(real_imgs, condition), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach(), condition), fake)
        d_loss = (real_loss + fake_loss) / 2
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], D_Loss: {:.4f}, G_Loss: {:.4f}'.format(
                epoch+1, num_epochs, i+1, len(train_loader), d_loss.item(), g_loss.item()))

## 8. Test

In [ ]:
generator.eval()
with torch.no_grad():
    for i in range(10):
        print(i)
        z = nn.init.normal(torch.Tensor(1,z_size),mean=0,std=0.1)
        labels = torch.LongTensor([i]).to(device)
        condition = torch.FloatTensor(1, 10).to(device)
        condition.zero_()
        labels = torch.unsqueeze(labels, 1).to(device)
        condition.scatter_(1, labels, 1)
        gen_imgs = generator(z, condition)
        gen_imgs = gen_imgs.view(1, 28, 28)
        plt.imshow(torch.squeeze(gen_imgs).numpy(), cmap='gray')
        plt.show()
generator.train()